In this project, we explore the dynamic world of online retail by working with a transactional dataset from a UK-based retailer, sourced from https://archive.ics.uci.edu/dataset/352/online+retail. The dataset includes all transactions recorded between 2010 and 2011 and provides a valuable foundation for understanding customer behavior.

Our goal is to strengthen marketing effectiveness and drive higher sales through customer segmentation. To achieve this, we first convert raw transactional data into a customer-focused dataset by engineering meaningful features. These features help us group customers into distinct segments using the K-means clustering algorithm, enabling deeper insights into customer preferences and behavioral patterns.

Once the customer segments are established, we extend the project by building a recommendation system. This system identifies the top-selling products within each segment and recommends them to customers who have not yet purchased those items. Through this approach, we aim to support targeted marketing strategies and enhance overall sales performance.

Objectives

• Data Cleaning & Transformation:
Clean the dataset by handling missing values, duplicates, and outliers to prepare it for effective clustering.

• Feature Engineering:
Create new features based on transactional data to build a customer-centric dataset for segmentation.

• Data Preprocessing:
Apply feature scaling and perform dimensionality reduction to streamline the data and improve clustering performance.

• Customer Segmentation using K-Means Clustering:
Segment customers into distinct groups using the K-Means algorithm to enable targeted marketing and personalized strategies.

• Cluster Analysis & Evaluation:
Analyze and profile each cluster to understand customer characteristics and assess the quality of the formed clusters.

• Recommendation System:
Build a system that recommends top-selling products to customers within each cluster who have not purchased those products yet, increasing sales and marketing effectiveness.

## Step 1 : Initialization

### Importing Necessary Libraries

In [4]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import plotly.graph_objects as go
from matplotlib.colors import LinearSegmentedColormap
from matplotlib import colors as mcolors
from scipy.stats import linregress
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from sklearn.cluster import KMeans
from tabulate import tabulate
from collections import Counter

%matplotlib inline

In [5]:
# Initialize Plotly for use in the notebook
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [6]:
# Configure Seaborn plot styles: Set background color and use dark grid
sns.set(rc={'axes.facecolor': '#fcf0dc'}, style='darkgrid')

### Step 1.2 - Loading the Dataset

In [11]:
import kagglehub
from pathlib import Path

# Download latest version (returns the folder path where files were extracted)
path = kagglehub.dataset_download("carrie1/ecommerce-data")

print("Path to dataset files:", path)

# Construct the path to the CSV file and load it into a DataFrame
csv_path = Path(path) / "data.csv"
print("Loading CSV:", csv_path)
df = pd.read_csv(csv_path, encoding="ISO-8859-1")
print("Loaded dataframe shape:", df.shape)
print("Columns:", df.columns.tolist())

Path to dataset files: C:\Users\utkar\.cache\kagglehub\datasets\carrie1\ecommerce-data\versions\1
Loading CSV: C:\Users\utkar\.cache\kagglehub\datasets\carrie1\ecommerce-data\versions\1\data.csv
Loaded dataframe shape: (541909, 8)
Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']
Loaded dataframe shape: (541909, 8)
Columns: ['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country']


| Variable     | Description                                                                 |
|--------------|-----------------------------------------------------------------------------|
| InvoiceNo    | Code representing each unique transaction. If it starts with 'C', it indicates a cancellation. |
| StockCode    | Code uniquely assigned to each distinct product.                            |
| Description  | Description of each product.                                                |
| Quantity     | The number of units of a product in a transaction.                          |
| InvoiceDate  | The date and time of the transaction.                                       |
| UnitPrice    | The unit price of the product in sterling.                                  |
| CustomerID   | Identifier uniquely assigned to each customer.                              |
| Country      | The country of the customer.                                                |

# Step 2 - Initial Data Analysis


### Step 2.1 - Dataset Overview

In [17]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [18]:
df.tail()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


Inferences:

The dataset contains 541,909 rows and 8 columns. Below is an overview of each variable:

InvoiceNo: A string-based field representing the invoice number for each transaction. A single invoice number may include multiple purchased items.

StockCode: A string column that stores the unique product code for each item.

Description: A text column containing product descriptions. It includes missing values, with 540,455 non-null entries.

Quantity: An integer field indicating how many units of a product were purchased.

InvoiceDate: A datetime field capturing the exact date and time of each transaction.

UnitPrice: A floating-point column representing the price per unit of each product.

CustomerID: A float column representing the unique identifier of each customer. This column has many missing values, with only 406,829 non-null entries.

Country: A string column specifying the customer’s country.

From this initial summary, we can see that the Description and CustomerID columns contain missing data that will require cleaning. The InvoiceDate field is already in an appropriate datetime format, making it convenient for time-based analysis. We can also observe that a single customer appears multiple times, indicating they have made several transactions.

The next steps include more detailed data cleaning, handling missing or incorrect entries, and engineering new features that support the project’s objectives.




### Step 2.2 - Summary Statistics

In [22]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,9.552250,218.081158,-80995.00,1.00,3.00,10.00,80995.0
UnitPrice,541909.0,4.611114,96.759853,-11062.06,1.25,2.08,4.13,38970.0
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.0


In [24]:
# Summary statistics for categorical variables
df.describe(include='object').T

,count,unique,top,freq
InvoiceNo,541909,25900,573585,1114
StockCode,541909,4070,85123A,2313
Description,540455,4223,WHITE HANGING HEART T-LIGHT HOLDER,2369
InvoiceDate,541909,23260,10/31/2011 14:41,1114
Country,541909,38,United Kingdom,495478


Inferences

Quantity:

The mean quantity of items per transaction is about 9.55.

The values span a very large range, from –80,995 to 80,995. Negative quantities represent returned or cancelled orders and will need careful handling.

The high standard deviation shows that the data is widely spread. The large gap between the maximum value and the 75th percentile indicates the presence of significant outliers.

UnitPrice:

The average unit price is approximately 4.61.

The range extends from –11,062.06 to 38,970, suggesting errors or noisy data because prices cannot logically be negative.

Similar to Quantity, the large difference between the maximum and the 75th percentile points to the existence of extreme outliers.

CustomerID:

There are 406,829 non-null CustomerID values, indicating missing entries that need to be managed.

CustomerID values range roughly from 12346 to 18287, allowing identification of unique customers.

InvoiceNo:

The dataset contains 25,900 unique invoice numbers, each representing a separate transaction.

The most frequently occurring invoice number is 573585, appearing 1,114 times, which may correspond to a large order or an invoice containing many individual items.

StockCode:

There are 4,070 unique stock codes, representing different products.

The most common stock code is 85123A, appearing 2,313 times.

Description:

The dataset includes 4,223 unique product descriptions.

The most frequent description is "WHITE HANGING HEART T-LIGHT HOLDER", appearing 2,369 times.

Some entries in this column are missing and will need to be handled during data cleaning.

Country:

The dataset includes transactions from 38 different countries.

A large majority—around 91.4%—of all transactions originate from the United Kingdom.